# Strength Score notebook 
## Functionallity:
* 1. Detect the dates that are relevant based on yahoo finanace
* 2. Create a strength score for those days. The score is 1 when the day is relevant and 0 when it is not relevant 

In [1]:
ticker='BTC-USD'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 7.3 MB/s 
     |████████████████████████████████| 62 kB 921 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import warnings
warnings.filterwarnings("ignore")
import yfinance as yfin
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff
import pandas as pd
import statsmodels.api as sm
from pandas_datareader import data as pdr
import statsmodels.api as sm
import eventstudy as es
from eventstudy import excelExporter
import numpy as np

See xlsxwriter's documentation: https://xlsxwriter.readthedocs.io/



In [3]:
# 1. Defining date range and downloading stock data from Yahoo Finance
start = "2018-03-29"
end = "2022-04-29"

stock_data = yf.download(ticker, start, end, adjusted=True)

stock_returns = stock_data['Adj Close'].resample('d').last().pct_change().dropna()
stock_returns.name = "Day_Rtn"

[*********************100%***********************]  1 of 1 completed


In [9]:
# For Google Collab:
#from google.colab import drive
#drive.mount('/content/drive')
#import pandas as pd
#ff3_daily = pd.read_csv('/content/drive/MyDrive/F-F_Research_Data_Factors_daily.CSV',parse_dates=['Unnamed: 0'])
#ff3_daily.reset_index(inplace=True)

In [18]:
# 2. Importing Fama French research dataset from Dartmouth and calculating expected return using OLS

ff3_daily=pd.read_csv("F-F_Research_Data_Factors_daily.CSV",parse_dates=['Unnamed: 0'])
ff3_daily.rename(columns={"Unnamed: 0": 'Date'}, inplace=True)
ff3_daily.set_index('Date', inplace=True)

ff_data = ff3_daily.merge(stock_returns,on='Date')

X = ff_data[['Mkt-RF', 'SMB', 'HML']]
y = ff_data['Day_Rtn'] - ff_data['RF']
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
intercept, b1, b2, b3 = ff_model.params

rf = ff_data['RF'].mean()
market_premium = ff3_daily['Mkt-RF'].mean()
size_premium = ff3_daily['SMB'].mean()
value_premium = ff3_daily['HML'].mean()

expected_daily_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium 

#ff3_daily = pd.read_csv('/content/drive/MyDrive/F-F_Research_Data_Factors_daily.CSV')
ff3_daily=pd.read_csv("F-F_Research_Data_Factors_daily.CSV")
ff3_daily.rename(columns = {'Unnamed: 0':'date'}, inplace = True)
ff3_daily.tail()

yfin.pdr_override()

stock_data = pdr.get_data_yahoo(ticker, start, end)

ff3_daily.to_csv("{}_famafrench.csv".format(ticker), index=False, date_format='%Y%m%d')

stock_data[ticker] = stock_data["Adj Close"].pct_change()
stock_data = stock_data.dropna()
stock_data.reset_index(level=0, inplace=True)
stock_data = stock_data[['Date',ticker]].copy()
stock_data.columns = ['date',ticker]
stock_data.to_csv("{}_returns.csv".format(ticker), index=False, date_format='%Y-%m-%d')

es.Single.import_FamaFrench("{}_famafrench.csv".format(ticker))
es.Single.import_returns("{}_returns.csv".format(ticker))

event = es.Single.FamaFrench_3factor(
    security_ticker = ticker,
    event_date = np.datetime64('2020-12-28'),
    event_window = (-2,+4), 
    estimation_size = 100,
    buffer_size = 30)

[*********************100%***********************]  1 of 1 completed


In [19]:
# 3. Using the relevant news from Step 1 of the project to conduct event study
#news3k = pd.read_csv('/content/drive/MyDrive/RelevanceScore.csv', encoding= 'unicode_escape')
news3k = pd.read_csv("RelevanceScore.csv", encoding= 'unicode_escape')
pd.set_option('display.max_colwidth', -1)
news3k["date_parsed"]=pd.to_datetime(news3k["date"])
news3k.head()

dates_for_eventstudy = news3k["date_parsed"].drop_duplicates().sort_values()

i = 1
listAR=[]
dates=[]
for date in dates_for_eventstudy:
    try:
        event = es.Single.FamaFrench_3factor(
            security_ticker = ticker,
            event_date = np.datetime64(date),
            event_window = (-2,+4), 
            estimation_size = 100,
            buffer_size = 30)
        listAR.append(event.AR)
        dates.append(date)
        
    except:
        list_nans = []
        for nans in range(event.event_window_size):
            list_nans.append(np.nan)
        listAR.append(list_nans)
        dates.append(date)
        
columns_ar=[]
for i in range(event.event_window[0],event.event_window[1]+1):
    columns_ar.append("AR"+str(i))

df_AR = pd.DataFrame(listAR, columns=columns_ar)
df_test2 = pd.DataFrame({'date':dates}).join(df_AR)
df_test2.tail()

df_test3 = df_test2.dropna()
df_test3["date"] = pd.to_datetime(df_test3["date"])
df_test3.tail()

n = 6  #6 is for semester

In [20]:
df_test3

,date,AR-2,AR-1,AR0,AR1,AR2,AR3,AR4
0,2022-04-06,0.023345,-0.012197,-0.047514,0.016851,-0.035267,0.007791,-0.018238
1,2022-04-07,-0.011748,-0.047104,0.016824,-0.035021,0.008332,-0.018010,-0.066699
2,2022-04-08,-0.047592,0.016230,-0.036427,0.008071,-0.018488,-0.067329,0.009638
3,2022-04-09,0.016327,-0.037015,0.009444,-0.017676,-0.067143,0.010548,0.018974
4,2022-04-10,-0.018997,0.019168,-0.018436,-0.066794,0.015087,0.023426,-0.023026
5,2022-04-11,0.018608,-0.005400,-0.072775,0.013665,0.033788,-0.035154,0.023173
6,2022-04-12,-0.008281,-0.072895,0.014372,0.036043,-0.035982,0.021001,0.010720
7,2022-04-19,-0.015839,0.026480,0.017001,-0.002962,-0.016395,-0.011979,-0.002361
8,2022-04-20,0.025900,0.017057,-0.003375,-0.016804,-0.012256,-0.002215,0.002829
9,2022-04-21,0.016913,-0.004023,-0.018145,-0.014810,-0.003170,0.001057,0.025184


In [21]:
# 4. Calculate the strength score - positive and negative

relevant_matrix_pos = df_test2.iloc[:,1:len(df_test3.columns)] > expected_daily_return*n 
relevant_matrix_neg = df_test2.iloc[:,1:len(df_test3.columns)] < -expected_daily_return*n

df_test3["Relevant_pos"] = relevant_matrix_pos[list(relevant_matrix_pos.columns)].any(axis=1).astype(int)
df_test3["Relevant_neg"] = -1*(relevant_matrix_neg[list(relevant_matrix_neg.columns)].any(axis=1).astype(int))

df_test3[(df_test3["Relevant_neg"] == -1) & (df_test3["Relevant_pos"] == 1)]

df_test3.date[(df_test3["Relevant_pos"]==1) | (df_test3["Relevant_pos"]==-1)]

news3k["date_parsed"]=pd.to_datetime(news3k['date_parsed'])
news3k_relevant = pd.merge(news3k,df_test3, left_on ="date_parsed", right_on="date").drop(columns=["date_parsed"],axis=1)

In [23]:
# 5. Filtering for dates with strength and generate csv file


news3k_relevant['Strength'] = np.where(((news3k_relevant["Relevant_pos"]==1) 
                                     & (news3k_relevant["Relevant_neg"]==0))| (
                        (news3k_relevant["Relevant_neg"]==-1) & (news3k_relevant["Relevant_pos"]==0)),1,0)
news3k_relevant

,Unnamed: 0,title,date_x,url,Relevance Score,coin,date_y,AR-2,AR-1,AR0,AR1,AR2,AR3,AR4,Relevant_pos,Relevant_neg,Strength
0,1,bitcoin : is bitcoin mining is legal ?,2022-04-10,https://www.marca.com/en/lifestyle/2022/04/10/62531631268e3ead338b4583.html,0.631420,bitcoin,2022-04-10,-0.018997,0.019168,-0.018436,-0.066794,0.015087,0.023426,-0.023026,0,-1,1
1,23,missed out on ethereum ? here what to buy now,2022-04-10,https://www.fool.com/investing/2022/04/10/missed-out-on-ethereum-heres-what-to-buy-now/?source=iedfolrf0000001,0.541291,ethereum,2022-04-10,-0.018997,0.019168,-0.018436,-0.066794,0.015087,0.023426,-0.023026,0,-1,1
2,24,missed out on ethereum ? here what to buy now,2022-04-10,https://www.fool.com/investing/2022/04/10/missed-out-on-ethereum-heres-what-to-buy-now/,0.541291,ethereum,2022-04-10,-0.018997,0.019168,-0.018436,-0.066794,0.015087,0.023426,-0.023026,0,-1,1
3,60,bitcoin cash price reaches $327 . 50 on exchanges ( bch ),2022-04-10,https://www.etfdailynews.com/2022/04/10/bitcoin-cash-price-reaches-327-50-on-exchanges-bch/,0.508527,bitcoin cash,2022-04-10,-0.018997,0.019168,-0.018436,-0.066794,0.015087,0.023426,-0.023026,0,-1,1
4,62,crypto cash out ! here who won bitcoin bonus money at ufc 273,2022-04-10,https://www.mmamania.com/2022/4/10/23019171/crypto-cash-out-heres-who-won-bitcoin-bonus-money-at-ufc-273,0.498178,bitcoin cash,2022-04-10,-0.018997,0.019168,-0.018436,-0.066794,0.015087,0.023426,-0.023026,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,249,bitcoin use as currency may just be getting started ( cryptocurrency : btc - usd ),2022-04-23,https://seekingalpha.com/news/3825613-bitcoins-use-as-currency-may-just-be-getting-started-morgan-stanley-says,0.309708,usd coin,2022-04-23,-0.016960,-0.012482,-0.001954,0.002386,0.025450,-0.059760,0.036457,1,-1,0
299,368,itwire - review â stellar data recovery premium,2022-04-23,https://itwire.com/reviews-sp-288/software/review-stellar-data-recovery-premium.html,0.380837,stellar,2022-04-23,-0.016960,-0.012482,-0.001954,0.002386,0.025450,-0.059760,0.036457,1,-1,0
300,378,newly - discovered stellar explosion the micronova could explain more on dead stars,2022-04-23,https://www.natureworldnews.com/articles/50503/20220422/stellar-explosion-the-micronova-dead-stars.htm,0.341086,stellar,2022-04-23,-0.016960,-0.012482,-0.001954,0.002386,0.025450,-0.059760,0.036457,1,-1,0
301,329,"cryptocurrencies price prediction : ethereum , ripple & bitcoin â american wrap 11 april",2022-04-12,https://www.fxstreet.com/cryptocurrencies/news/cryptocurrencies-price-prediction-ethereum-ripple-bitcoin-american-wrap-11-april-202204112234,0.381453,ripple,2022-04-12,-0.008281,-0.072895,0.014372,0.036043,-0.035982,0.021001,0.010720,1,-1,0


In [ ]:
news3k_relevant.to_csv('strength_output.csv', index = False)